In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/reversible///')
%cd /home/schirrmr/


In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png' 
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

In [ ]:
from numpy.random import RandomState
from braindecode.torch_ext.util import np_to_var, var_to_np
from IPython.display import display

In [ ]:
x = np_to_var([-1.5,1.5], dtype=np.float32, requires_grad=True)
valid_x = np_to_var([-0.5, 0.5], dtype=np.float32, requires_grad=True)
x_fake = np_to_var(np.linspace(-2,2, 400), dtype=np.float32, requires_grad=True)
plt.scatter(var_to_np(x),  var_to_np(x) * 0, )
plt.scatter(var_to_np(valid_x),  var_to_np(valid_x) * 0, )
plt.scatter(var_to_np(x_fake),  var_to_np(x_fake) * 0 - 0.1, alpha=0.2, marker='x')

In [ ]:
from torch.nn import ConstantPad2d
import torch as th
from torch import nn

In [ ]:
from reversible2.plot import display_close
def get_prob(x, dists):
    return th.mean(th.stack([th.exp(dist.log_prob(x)) for dist in dists]), dim=0)

In [ ]:
from reversible2.sliced import sliced_from_samples

In [ ]:
sliced_from_samples(samples.unsqueeze(1), x_fake.unsqueeze(1), 1, adv_dirs=None)

In [ ]:
%%time

In [ ]:
%%time
ot_euclidean_loss_for_samples(samples.unsqueeze(1), x_fake.unsqueeze(1))

In [ ]:
from reversible2.ot_exact import ot_euclidean_loss_for_samples
all_x = th.cat((x, valid_x))
x_fake = np_to_var(np.linspace(-2,2, 400), dtype=np.float32, requires_grad=True)


log_stds = th.zeros(4, requires_grad=True)

dists = [th.distributions.Normal(m, s * s)
    for m, s in zip(th.cat((x, valid_x), dim=0),
                    th.exp(log_stds))]

optimizer = th.optim.Adam([log_stds], lr=1e-2)
optim_fake_x = th.optim.Adam([x_fake], lr=1e-3)

In [ ]:
n_epochs = 2000
for i_epoch in range(n_epochs):
    dists = [th.distributions.Normal(m, s * s)
        for m, s in zip(th.cat((x, valid_x), dim=0),
                        th.exp(log_stds))]


    likelihoods = [get_prob(all_x[i_x], [dists[i_dist] for i_dist in np.setdiff1d(np.arange(4), [i_x])])
                            for i_x in range(4)]
    loss = -th.mean(th.stack(likelihoods))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    samples = th.cat([d.sample([100]) for d in dists])
    #ot_loss = ot_euclidean_loss_for_samples(samples.unsqueeze(1), x_fake.unsqueeze(1))
    # same as ot loss, much faster
    ot_loss = sliced_from_samples(samples.unsqueeze(1), x_fake.unsqueeze(1), 1, adv_dirs=None,
                   dist=lambda diffs:th.mean(th.abs(diffs)))
    optim_fake_x.zero_grad()
    ot_loss.backward()
    optim_fake_x.step()
    
    
    if i_epoch % (n_epochs // 20) == 0:
        test_grid_x = th.linspace(-6,6,1000)

        probs = get_prob(test_grid_x, dists)
        fig = plt.figure(figsize=(8,3))
        plt.plot(var_to_np(test_grid_x), var_to_np(probs))

        for i_dist in range(4):
            probs = get_prob(test_grid_x, dists[i_dist:i_dist+1])
            plt.plot(var_to_np(test_grid_x), var_to_np(probs), color=seaborn.color_palette()[1],
                    lw=0.5)
        ax = plt.gca()
        plt.scatter(var_to_np(x_fake),  var_to_np(x_fake) * 0 - 0.1, alpha=0.2, marker='x')
        seaborn.distplot(var_to_np(x_fake), color=seaborn.color_palette()[2], rug=True)
        display_close(fig)